## Importing libraries and argparse

In [1]:
import torch
import os
import cv2
from utils.loading_utils import load_model, get_device
import argparse
from utils.event_readers import FixedSizeEventReader, FixedDurationEventReader
from utils.inference_utils import events_to_voxel_grid, events_to_voxel_grid_pytorch
import numpy as np
from utils.timers import Timer
from image_reconstructor import ImageReconstructor
from options.inference_options import set_inference_options


def main_inference_options():
    parser = argparse.ArgumentParser(description="Evaluating a trained network")
    parser.add_argument(
        "-c", "--path_to_model", required=False, type=str, help="path to model weights"
    )
    parser.add_argument("-i", "--input_file", required=False, type=str)
    parser.add_argument("--fixed_duration", dest="fixed_duration", action="store_true")
    parser.set_defaults(fixed_duration=False)
    parser.add_argument(
        "-N",
        "--window_size",
        default=None,
        type=int,
        help="Size of each event window, in number of events. Ignored if --fixed_duration=True",
    )
    parser.add_argument(
        "-T",
        "--window_duration",
        default=33.33,
        type=float,
        help="Duration of each event window, in milliseconds. Ignored if --fixed_duration=False",
    )
    parser.add_argument(
        "--num_events_per_pixel",
        default=0.35,
        type=float,
        help="in case N (window size) is not specified, it will be \
                              automatically computed as N = width * height * num_events_per_pixel",
    )
    parser.add_argument("--skipevents", default=0, type=int)
    parser.add_argument("--suboffset", default=0, type=int)
    parser.add_argument(
        "--compute_voxel_grid_on_cpu",
        dest="compute_voxel_grid_on_cpu",
        action="store_true",
    )
    parser.set_defaults(compute_voxel_grid_on_cpu=False)

    return parser

### Rewrite data reader to avoid file IO and $O(n^2)$ event window iterations in original code of E2VID, or code infinitely hangs up on large GOPRO datasets. No changes to logics and outputs.

In [2]:
import numpy as np
from utils.timers import Timer


class MyFixedSizeEventReader:
    """
    Reads events from a '.h5' file, and packages the events into
    non-overlapping event windows, each containing a fixed number of events.
    """

    def __init__(self, events, num_events=10000, start_index=0):
        print("Will use fixed size event windows with {} events".format(num_events))
        print("Output frame rate: variable")
        self.events = events[start_index:]
        self.num_events = num_events

    def __iter__(self):
        return self

    def __next__(self):
        if len(self.events) > self.num_events:
            event_window = self.events[: self.num_events]
            self.events = self.events[self.num_events :]
        else:
            event_window = self.events[:]
            self.events = []
        event_window = np.array(event_window)
        return event_window


import numpy as np


class MyFixedDurationEventReader:
    """
    Reads events from a '.h5' file, and packages the events into
    non-overlapping event windows, each of a fixed duration.
    """

    def __init__(self, events, duration_ms=50.0, start_index=0):
        print(
            "Will use fixed duration event windows of size {:.2f} ms".format(
                duration_ms
            )
        )
        print("Output frame rate: {:.1f} Hz".format(1000.0 / duration_ms))
        self.events = np.array(events[start_index:])
        self.events = self.events[
            self.events[:, 0].argsort()
        ]  # Sort the events based on time
        self.duration_s = duration_ms / 1000.0
        self.current_index = 0

    def __iter__(self):
        return self

    def __del__(self):
        self.event_file.close()

    def __next__(self):
        if self.current_index >= len(self.events):
            raise StopIteration
        start_time = self.events[self.current_index][0]
        end_index = self.events[:, 0].searchsorted(
            start_time + self.duration_s, side="right"
        )
        event_list = self.events[self.current_index : end_index]
        self.current_index = end_index
        return event_list

### Dynamic reader for alignment

In [3]:
class GroundTruthTimestampEventReader:
    """
    Reads events from a '.h5' file, and packages the events into
    event windows whose start and end times align with the timestamps of the ground truth images.
    """

    def __init__(self, events, ground_truth_timestamps, start_index=0):
        self.events = np.array(events[start_index:])
        self.events = self.events[self.events[:, 0].argsort()]  # Sort the events based on time
        self.ground_truth_timestamps = np.array(ground_truth_timestamps) / 1e9
        self.current_index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_index >= len(self.ground_truth_timestamps):
            raise StopIteration
        start_time = self.ground_truth_timestamps[self.current_index]
        end_time = self.ground_truth_timestamps[self.current_index + 1] if self.current_index + 1 < len(self.ground_truth_timestamps) else self.events[-1, 0]
        start_index = self.events[:, 0].searchsorted(start_time, side='left')
        end_index = self.events[:, 0].searchsorted(end_time, side='right')
        event_list = self.events[start_index:end_index]
        self.current_index += 1
        return event_list


In [4]:
import h5py
import pandas as pd

def convert_h5_to_df(h5_file_path):
    with h5py.File(h5_file_path, 'r') as f:
        events = f['events']
        xs = events['xs'][:]
        ys = events['ys'][:]
        ts = events['ts'][:]
        ps = events['ps'][:]
        
    df = pd.DataFrame({
        't': (ts / 1e9).astype(np.float64), # In seconds
        'x': xs.astype(np.int16), 
        'y': ys.astype(np.int16),
        'pol': ps.astype(np.int16)
    })
    
    return df

def convert_h5_to_txt(h5_file_path, txt_file_path):
    with h5py.File(h5_file_path, 'r') as f:
        events = f['events']
        xs = events['xs'][:]
        ys = events['ys'][:]
        ts = events['ts'][:]
        ps = events['ps'][:]
        height, width = f.attrs['sensor_resolution'][0:2]
    
    # Create a pandas DataFrame
    df = pd.DataFrame({
        'ts': ts,
        'xs': xs,
        'ys': ys,
        'ps': ps
    })

    with open(txt_file_path, 'w') as txt_file:
        print(f"Writing to {txt_file_path}")
        # Write header
        txt_file.write(f"{width} {height}\n")  
    
    # Append DataFrame contents to the file (after the header)
    df.to_csv(txt_file_path, sep=' ', index=False, header=False, mode='a')


TXT_STRUCTURE = "ptxy"  # Set this variable according to your requirement

def convert_txt_to_df(txt_file_path):
    with open(txt_file_path, 'r') as f:
        lines = f.readlines()

    data = {'t': [], 'x': [], 'y': [], 'pol': []}
    for line in lines[1:]:  # Ignore the first line which is the sensor info
        parts = line.split()

        # Depending on your data structure 'txyp' or 'pxyt'
        if TXT_STRUCTURE == 'txyp':
            data['t'].append(float(parts[0]))
            data['x'].append(int(parts[1]))
            data['y'].append(int(parts[2]))
            data['pol'].append(int(parts[3]))
        elif TXT_STRUCTURE == 'pxyt':
            data['t'].append(float(parts[1]))
            data['x'].append(int(parts[2]))
            data['y'].append(int(parts[3]))
            data['pol'].append(int(parts[0]))
        elif TXT_STRUCTURE == 'ptxy':
            data['t'].append(float(parts[1]))
            data['x'].append(int(parts[2]))
            data['y'].append(int(parts[3]))
            data['pol'].append(int(parts[0]))
        else:
            raise ValueError(f"Unknown TXT_STRUCTURE: {TXT_STRUCTURE}")

    # Create DataFrame
    df = pd.DataFrame({
        't': np.array(data['t'], dtype=np.float64),
        'x': np.array(data['x'], dtype=np.int16),
        'y': np.array(data['y'], dtype=np.int16),
        'pol': np.array(data['pol'], dtype=np.int16)
    })

    return df



In [5]:
EXTRACT_COLORED = False


def extract_gt_images(h5_file_path, output_folder):
    """
    Extracts ground truth images from an h5 file and saves them to a specified output folder.

    Args:
    - h5_file_path (str): The path to the h5 file containing the ground truth images.
    - output_folder (str): The path to the folder where the extracted images will be saved.

    Returns:
    - avg_interval (float): The average time interval between the extracted images.
    """
    print(f"Extracting ground truth images from {h5_file_path} to {output_folder}")
    # Create gt folder if it doesn't exist
    gt_folder = os.path.join(output_folder, 'gt')
    os.makedirs(gt_folder, exist_ok=True)
    
    timestamps = [] # In ns
    with h5py.File(h5_file_path, 'r') as f:
        images_group = f['sharp_images']
        for image_name in images_group:
            image = images_group[image_name][:]

            timestamp = images_group[image_name].attrs['timestamp']
            timestamps.append(timestamp)  # store timestamps

            image_path = os.path.join(gt_folder, f"image_{timestamp:.0f}.png")
            if EXTRACT_COLORED:
                cv2.imwrite(image_path, image)
            else:
                cv2.imwrite(image_path, image[:,:,0])
    
    timestamps = np.array(timestamps)
    intervals = np.diff(timestamps)
    avg_interval = np.mean(intervals) / 1e6  # in ms
    std_interval = np.std(intervals) / 1e6  # in ms

    # Save timestamps to a timestamps.txt in seconds, each line is a timestamp
    timestamps = timestamps / 1e9  # in seconds
    timestamps_path = os.path.join(gt_folder, 'timestamps.txt')
    np.savetxt(timestamps_path, timestamps, fmt='%.10f')

    return avg_interval, std_interval

### Start processing

In [6]:
USE_DYNAMIC_WINDOWING = True
def process_events(args):
    if args.width and args.height:  # sensor size is provided in args
        width = args.width
        height = args.height
    else:  # read sensor size from the first line of the event file
        path_to_events = args.input_file
        # Raise an error if the file does not exist
        if not os.path.exists(path_to_events):
            raise ValueError(f"File {path_to_events} does not exist")
        
        # Determine the type of the file
        _, file_extension = os.path.splitext(args.input_file)
        if file_extension == '.txt':
            header = pd.read_csv(path_to_events, delim_whitespace=True, header=None, names=['width', 'height'],
                                dtype={'width': np.int, 'height': np.int},
                                nrows=1)
            width, height = header.values[0]
        elif file_extension == '.h5':
            with h5py.File(path_to_events, 'r') as f:
                height, width = f.attrs['sensor_resolution'][0:2]
        else:
            raise ValueError(f"Unknown file extension {file_extension}")
    
        print('Sensor size: {} x {}'.format(width, height))

    # Load model
    model = load_model(args.path_to_model)
    device = get_device(args.use_gpu)

    model = model.to(device)
    model.eval()
    
    reconstructor = ImageReconstructor(model, height, width, model.num_bins, args)

    N = args.window_size
    if not args.fixed_duration:
        if N is None:
            N = int(width * height * args.num_events_per_pixel)
            print('Will use {} events per tensor (automatically estimated with num_events_per_pixel={:0.2f}).'.format(
                N, args.num_events_per_pixel))
        else:
            print('Will use {} events per tensor (user-specified)'.format(N))
            mean_num_events_per_pixel = float(N) / float(width * height)
            if mean_num_events_per_pixel < 0.1:
                print('!!Warning!! the number of events used ({}) seems to be low compared to the sensor size. \
                    The reconstruction results might be suboptimal.'.format(N))
            elif mean_num_events_per_pixel > 1.5:
                print('!!Warning!! the number of events used ({}) seems to be high compared to the sensor size. \
                    The reconstruction results might be suboptimal.'.format(N))

    initial_offset = args.skipevents
    sub_offset = args.suboffset
    start_index = initial_offset + sub_offset

    if args.compute_voxel_grid_on_cpu:
        print('Will compute voxel grid on CPU.')

    if USE_DYNAMIC_WINDOWING:
        timestamps_path = os.path.join(args.output_folder, 'gt', 'timestamps.txt')
        timestamps = np.loadtxt(timestamps_path)
        timestamps = timestamps * 1e9  # Convert to ns
        if file_extension == '.h5':
            df = convert_h5_to_df(args.input_file)
        elif file_extension == '.txt':
            df = convert_txt_to_df(args.input_file)
        else:
            raise ValueError(f"Dynamic windowing is only supported for h5 files. Got {file_extension}")
        event_window_iterator = GroundTruthTimestampEventReader(df.values, timestamps)

    else:
        if args.fixed_duration:
            if file_extension == '.h5':
                # Convert h5 to pandas DataFrame
                df = convert_h5_to_df(args.input_file)
                event_window_iterator = MyFixedDurationEventReader(df.values,
                                                                duration_ms=args.window_duration)
            else:  # Assuming txt
                event_window_iterator = FixedDurationEventReader(path_to_events,
                                                                duration_ms=args.window_duration,
                                                                start_index=start_index)
        else:
            if file_extension == '.h5':
                # Convert h5 to pandas DataFrame
                df = convert_h5_to_df(args.input_file)
                event_window_iterator = MyFixedSizeEventReader(df.values, num_events=N)
            else:  # Assuming txt
                event_window_iterator = FixedSizeEventReader(path_to_events, num_events=N, start_index=start_index)

    with Timer('Processing entire dataset'):
        for event_window in event_window_iterator:
            # Check if event_window is empty
            if event_window.size == 0:
                break
            
            last_timestamp = event_window[-1, 0]

            with Timer('Building event tensor'):
                if args.compute_voxel_grid_on_cpu:
                    event_tensor = events_to_voxel_grid(event_window,
                                                        num_bins=model.num_bins,
                                                        width=width,
                                                        height=height)
                    event_tensor = torch.from_numpy(event_tensor)
                else:
                    event_tensor = events_to_voxel_grid_pytorch(event_window,
                                                                num_bins=model.num_bins,
                                                                width=width,
                                                                height=height,
                                                                device=device)

            num_events_in_window = event_window.shape[0]
            reconstructor.update_reconstruction(event_tensor, start_index + num_events_in_window, last_timestamp)

            start_index += num_events_in_window
    reconstructor.image_writer.timestamps_file.close()

In [2]:
import glob
from PIL import Image


def process_dataset(
    dataset_dir, path_to_model, use_gpu, output_folder, use_resblur=False
):
    parser = main_inference_options()
    set_inference_options(parser)
    default_args = vars(parser.parse_args([]))  # get a dictionary of defaults
    default_args.update(
        {
            "use_gpu": use_gpu,
            "auto_hdr": True,
            "fixed_duration": True,
        }
    )
    os.makedirs(output_folder, exist_ok=True)
    for split in os.listdir(dataset_dir):
        split_path = os.path.join(dataset_dir, split)
        if use_resblur:
            pass
            # if os.path.isdir(split_path):
            #     # for folder in os.listdir(split_path):
            #     #     folder_path = os.path.join(split_path, folder)
            #     # Assert eventcntts_hhfcode_unnormed0606_txt and frame_clip folder exists, or AssertError
            #     assert "eventcntts_hhfcode_unnormed0606_txt" in os.listdir(
            #         split_path
            #     ) and "frame_clip" in os.listdir(split_path)
            #     events_path = os.path.join(
            #         split_path, "eventcntts_hhfcode_unnormed0606_txt"
            #     )
            #     ground_truth_path = os.path.join(split_path, "frame_clip")
            #     gt_path = os.path.join(output_folder, split, "gt")

            #     # Read all txt by order of filename and concat into an object, later load into Dataframe
            #     txt_files = sorted(glob.glob(os.path.join(events_path, "*.txt")))
            #     concatenated = ""
            #     for txt_file in txt_files:
            #         with open(txt_file, "r") as f:
            #             concatenated += f.read()

            #     # Save the concatenated txt to a new txt file in split_path
            #     concatenated_txt_path = os.path.join(split_path, "concatenated.txt")
            #     with open(concatenated_txt_path, "w") as f:
            #         f.write(concatenated)

            #     # Parse all file names like [18707981-18710981]_000002.png, which means the start and end timestamps of the frame exposure in ns. Compute The average time interval between the extracted images.(Only start of exposure). And save the timestamps to a timestamps.txt in seconds, each line is a timestamp. And also save the image files to the gt folder, in the filename of image_{timestamp}.png
            #     timestamps = []
            #     for image_name in os.listdir(ground_truth_path):
            #         image_path = os.path.join(ground_truth_path, image_name)
            #         timestamp = int(image_name.split("_")[0].split("-")[0])
            #         timestamps.append(timestamp)
            #         image_path = os.path.join(gt_path, f"image_{timestamp}.png")

            #         image = cv2.imread(image_path)
            #         cv2.imwrite(image_path, image[:, :, 0])

            #     timestamps = np.array(timestamps)
            #     intervals = np.diff(timestamps)
            #     avg_interval = np.mean(intervals) / 1e6  # in ms
            #     std_interval = np.std(intervals) / 1e6  # in ms
            #     print(f"Average frame interval: {avg_interval}, std: {std_interval}")

        else:
            # Check if subfolder is named train or test
            if os.path.isdir(split_path) and split in ["train", "test"]:
                for file in os.listdir(split_path):
                    folder_path = os.path.join(split_path, file)
                    subdir_output_folder = os.path.join(
                        output_folder, split, file.split(".")[0]
                    )
                    os.makedirs(subdir_output_folder, exist_ok=True)

                    if file.endswith(".h5"):  # convert .h5 files to .txt
                        # print(f"Converting {file_path} to .txt")

                        # txt_file_path = os.path.join(split_path, file.split('.')[0] + '.txt')
                        # convert_h5_to_txt(file_path, txt_file_path)
                        avg_frame_interval, std_frame_interval = extract_gt_images(
                            folder_path, subdir_output_folder
                        )
                        print(
                            f"Average frame interval: {avg_frame_interval}, std: {std_frame_interval}"
                        )
                        # file_path = txt_file_path  # use the converted txt file

                    elif not file.endswith(".txt"):  # skip non-txt files
                        continue

                    print(f"Processing {folder_path}")

                    args = default_args.copy()  # start with defaults
                    # update with specific settings for this file
                    args.update(
                        {
                            "path_to_model": path_to_model,
                            "input_file": folder_path,
                            "use_gpu": use_gpu,
                            "output_folder": subdir_output_folder,
                            "window_duration": avg_frame_interval,
                        }
                    )
                    args_namespace = argparse.Namespace(**args)
                    process_events(args_namespace)

def get_sensor_size_from_image(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height


In [ ]:
def process_resblur_scenario(scenario_path, path_to_model, use_gpu, output_folder):
    parser = main_inference_options()
    set_inference_options(parser)
    default_args = vars(parser.parse_args([]))  # get a dictionary of defaults
    default_args.update(
        {
            "use_gpu": use_gpu,
            "auto_hdr": True,
            "fixed_duration": True,
        }
    )
    os.makedirs(output_folder, exist_ok=True)
    
    events_txt_path = os.path.join(scenario_path, 'events_txt')
    frame_clip_path = os.path.join(scenario_path, 'frame_clip')
    
    # Get sensor size from a ground truth image
    gt_image_path = next(glob.glob(os.path.join(frame_clip_path, '*.png')))
    width, height = get_sensor_size_from_image(gt_image_path)
    
    # Concatenate all event text files into a DataFrame
    txt_files = sorted(glob.glob(os.path.join(events_txt_path, '*.txt')))
    events_df = pd.concat((convert_txt_to_df(f) for f in txt_files), ignore_index=True)
    
    # Process events
    print(f"Processing {scenario_path}")
    
    subdir_output_folder = os.path.join(output_folder, os.path.basename(scenario_path))
    os.makedirs(subdir_output_folder, exist_ok=True)

    args = default_args.copy()  # start with defaults
    # update with specific settings for this scenario
    args.update(
        {
            "path_to_model": path_to_model,
            "input_file": events_df,  # pass DataFrame directly
            "use_gpu": use_gpu,
            "output_folder": subdir_output_folder,
            "window_duration": 33.33,  # you may need to adjust this value
            "width": width,
            "height": height,
        }
    )
    args_namespace = argparse.Namespace(**args)
    process_events(args_namespace)

## Run data preparation and reconstruction in E2VID

### For EFNET

In [ ]:
if __name__ == "__main__":
    dataset_dir = "/root/autodl-tmp/datasets/GOPRO_rawevents"
    path_to_model = '/root/autodl-tmp/rpg_e2vid/pretrained/E2VID_lightweight.pth.tar'
    use_gpu = True
    output_folder = "/root/autodl-tmp/datasets/GOPRO_e2vid_pred"

    process_dataset(dataset_dir, path_to_model, use_gpu, output_folder)


### For RESBlur

In [ ]:
if __name__ == "__main__":
    dataset_dir = "RESBLUR"
    path_to_model = '/root/autodl-tmp/rpg_e2vid/pretrained/E2VID_lightweight.pth.tar'
    use_gpu = True
    output_folder = "RESBLUR_OUT"

    for scenario in os.listdir(dataset_dir):
        scenario_path = os.path.join(dataset_dir, scenario)
        if os.path.isdir(scenario_path):
            process_resblur_scenario(scenario_path, path_to_model, use_gpu, output_folder)